In [1]:

import pandas as pd
import numpy as np
import featuretools as ft
from sklearn.model_selection import train_test_split
# Load the rock mines dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data"
df = pd.read_csv(url)
column_names = ["sensor_" + str(i) for i in range(1, 61)] + ["target"]
df.columns = column_names
df['target'] = df['target'].map({'M': 1, 'R': 0})
x_data = df.iloc[:, :60]
y_data = df['target']
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=0) 
es_train = ft.EntitySet(id="Sonar_train")
es_train = es_train.add_dataframe(
    dataframe_name="Sonar",
    dataframe=x_train,
    index="id",
    make_index=True
    )
es_test = ft.EntitySet(id="Sonar_test")
es_test = es_test.add_dataframe(
    dataframe_name="Sonar",
    dataframe=x_test,
    index="id",
    make_index=True
    )
#get overview with
#ft.list_primitives()
feature_matrix, feature_defs = ft.dfs(
    entityset=es_train, target_dataframe_name="Sonar",
    trans_primitives=["add_numeric", "multiply_numeric", "subtract_numeric", "divide_numeric"],
    max_depth=3
)
### Save Features into File
feature_matrix_enc, features_enc = ft.encode_features(feature_matrix, feature_defs)
ft.save_features(features_enc, "feature_definitions.json")
### Load Features and apply them on test set
saved_features = ft.load_features("feature_definitions.json")
feature_matrix_test = ft.calculate_feature_matrix(saved_features, es_test)
## Create new data with target
x_train_enc = feature_matrix
x_train_enc.replace([np.inf, -np.inf], np.nan, inplace=True)
x_train_enc.dropna(axis='columns', how='any', inplace=True)
x_test_enc = feature_matrix_test
x_test_enc.dropna(axis='columns', how='any', inplace=True)





c:\Users\LArns\Desktop\meica\.conda\Lib\site-packages\featuretools\synthesis\deep_feature_synthesis.py:169: UserWarning: Only one dataframe in entityset, changing max_depth to 1 since deeper features cannot be created
  warnings.warn(
